## Explaining the Demand & Expected Earnings

Airbnb has successfully disrupted the traditional hospitality industry as more and more travelers decide to use Airbnb as their primary accommodation provider. Since its inception in 2008, Airbnb has seen an enormous growth, with the number of rentals listed on its website growing exponentially each year.

In Germany, no city is more popular than Berlin. That implies that Berlin is one of the hottest markets for Airbnb in Europe, with over 22,552 listings as of November 2018. With a size of 891 km², this means there are roughly 25 homes being rented out per km² in Berlin on Airbnb!

Considering the possibility that I might have to relocate for a new data science job, but want to keep my current flat in Berlin (which is quite cheap!), I might wonder if it could be worth it to offer my jewel on Airbnb. Could this perhaps be a profitable option? However, it is difficult for potential hosts to know what the true value of their home is, and how in-demand their home might be. And since location and furniture are obviously fixed for the most part, is there anything else a host can influence - such as description, communication patterns, and/or additional services to boost their earnings?

The following question will drive this project:

> **How big is the demand likely to be? What can a host expect with respect to occupancy and earnings here in Berlin? What factors influence how in-demand it is?**

### The datasets

In this notebook, I will combine the summary listings and the detailed Berlin listings data, sourced from the Inside Airbnb website. The goal is to visualize the rental landscape and to try to understand the aspects influencing the demand. Both datasets were scraped on November 07th, 2018.

## Table of Contents
<a id='Table of contents'></a>

### <a href='#1. Obtaining and Viewing the Data'> 1. Obtaining and Viewing the Data </a>

### <a href='#2. Preprocessing the Data'> 2. Preprocessing the Data </a>
* <a href='#2.1. Engineer Reviews per Month'> 2.1. Engineer Reviews per Month </a>
* <a href='#2.2. Combining Datasets'> 2.2. Combining Datasets </a>
* <a href='#2.3. Cleaning the Price Columns'> 2.3. Cleaning the Price Columns </a>
* <a href='#2.4. Occupancy Estimate'> 2.4. Occupancy Estimate </a>
* <a href='#2.5. Income Estimate'> 2.5. Income Estimate </a>

### <a href='#3. Exploratory Data Analysis (EDA)'> 3. Exploratory Data Analysis (EDA) </a>
* <a href='#3.1. Getting Started with a Heat Map'> 3.1. Getting Started with a Heat Map </a>
* <a href='#3.2. Seasonal Demand'> 3.2. Seasonal Demand </a>
* <a href='#3.3. Demand by Status SUPERHOST'> 3.3. Demand by Status SUPERHOST </a>
* <a href='#3.4. Demand by Neighbourhood'> 3.4. Demand by Neighbourhood </a>
* <a href='#3.5. Demand by Capacity'> 3.5. Demand by Capacity </a>

* <a href='#3.6. Demand by Price'> 3.6. Demand by Price </a>

### <a href='#4. Interpreting the Data'> 4. Interpreting the Data </a>

### <a href='#5. Appendix'> 5. Appendix </a>

### 1. Obtaining and Viewing the Data 
<a id='1. Obtaining and Viewing the Data'></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns
import folium

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_raw = pd.read_csv('../input/reviews_summary.csv')
df_raw.head(2)

In [ ]:
# drop unnecessary columns
df_raw.drop(['reviewer_id', 'reviewer_name', 'comments'], axis=1, inplace=True)

# rename id column and replace current value simply with 1 to later count the # of reviews per accommodation
df_raw.rename({'id':'review_count'}, axis=1, inplace=True)
df_raw.review_count = 1

# checking shape and duplicates
print("The first dataset has {} rows and {} columns.".format(*df_raw.shape))
print("It contains {} duplicates.".format(df_raw.duplicated().sum()))

df_raw.head(2)

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 2. Preprocessing the Data 
<a id='2. Preprocessing the Data'></a>

#### 2.1. Engineer Reviews per Month
<a id='2.1. Engineer Reviews per Month'></a>

*1. Process the date column and create a new dataframe*

In [ ]:
df_raw.info()

In [ ]:
# cast date-column from current object datatype to datetime datatype
df_raw.date = pd.to_datetime(df_raw.date, format="%Y-%m-%d")

Let's take the most consecutive 12 months we can get a hold of. As the scraping took place on November 07th 2018, we can have a full year starting November 1st, 2017 and ending October 31st, 2018:

In [ ]:
count_2017_18 = df_raw[df_raw["date"].isin(pd.date_range('2017-11-01', '2018-10-31'))]
print("The dataset 2017/2018 has {} reviews and {} columns.".format(*count_2017_18.shape))

In [ ]:
# set the date as index and sort by index
count_2017_18 = count_2017_18.set_index('date').sort_index()

*2. Calculate reviews per month*

In [ ]:
# group by month and listing_id, i.e. specific apartment 
# agg argument: #work on this col:  {#name the new col : #perform count operation}                        
count_2017_18_monthly = count_2017_18.groupby([pd.TimeGrouper(freq='M'), 'listing_id'])['review_count']\
                                         .agg({'review_count':{'reviews_per_month':'count'}})\
                                         .reset_index()

In [ ]:
# initial check
count_2017_18_monthly.head(2)

In [ ]:
# to flatten the multi-index, we a) reset the index in the groupby statement above ...
# ... and b) now define column names
count_2017_18_monthly.columns = ['date', 'listing_id', 'reviews_per_month']

# set date as index again
count_2017_18_monthly.set_index('date', inplace=True)

# check again
count_2017_18_monthly.head(2)

In [ ]:
#count_2017_18_monthly[count_2017_18_monthly.listing_id==2015]

In [ ]:
# get rid of date index
count_2017_18_monthly = count_2017_18_monthly.reset_index()

# group by listing_id and count reviews, turn it into a dataframe without groupby index
count_2017_18_monthly = pd.DataFrame(count_2017_18_monthly.groupby('listing_id')['reviews_per_month'].mean()\
                                                          .reset_index())

count_2017_18_monthly.head()

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.2. Combining Datasets
<a id='2.2. Combining Datasets'></a>

So far we only have a simple count of reviews per lodging. Let's use a second dataset to enrich our data:

In [ ]:
df_2 = pd.read_csv('../input/listings_summary.csv')
df_2.head(1)

In [ ]:
# merge full 2017/2018 dataframe + add specific columns from df_2
df = pd.merge(count_2017_18_monthly, df_2[['id', 'price', 'cleaning_fee', 'neighbourhood_group_cleansed', 
                          'latitude', 'longitude', 'property_type', 'accommodates', 
                          'bathrooms', 'bedrooms', 'bed_type', 'host_is_superhost']], 
              left_on='listing_id', right_on='id')

# drop 'id' from right dataframe as it's the same as 'listing_id' on the left dataframe
df.drop(['id'], axis=1, inplace=True)

# shorten the name of one column
df.rename({'neighbourhood_group_cleansed':'neighbourhood'}, axis=1, inplace=True)

# check the dataframe
print("The dataset has {} rows and {} columns - after combining it.".format(*df.shape))
print("It contains {} duplicates.".format(df.duplicated().sum()))
df.head(2)

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.3. Cleaning the Price Columns
<a id='2.3. Cleaning the Price Columns'></a>

In [ ]:
df.info()

As we can see above, `price` and `cleaning_fee` are both string columns, with the latter containing substantial null values that most likely mean that these hoste do not charge cleaning fees. Let's quickly tidy up here:

In [ ]:
# convert price column into numeric value
df.price = df.price.str.replace('$', '').str.replace(',', '').astype(float).astype(int)
df.price.isna().sum()

In [ ]:
# replace missing values with zero and convert cleaning_fee column into numeric value
df.cleaning_fee.fillna('$0.00', inplace=True)
df.cleaning_fee = df.cleaning_fee.str.replace('$', '').str.replace(',', '').astype(float).astype(int)
df.cleaning_fee.isna().sum()

In [ ]:
# investigate price column
df['price'].describe()

In [ ]:
# boxplot of price column
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
df['price'].plot(kind='box', xlim=(0, 1000), vert=False, flierprops=red_square, figsize=(16,2));

The boxplot above shows quite a skewed distribution with a long tail of high-priced outliers. However, 75% of all rentals only cost up to 70 Euro. For this project, let's remove the extremely high priced rentals above € 300/night to maintain comparability.

In [ ]:
df.drop(df[df['price'] > 300].index, axis=0, inplace=True)
df['price'].describe()

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.4. Occupancy Estimate
<a id='2.4. Occupancy Estimate'></a>

One of the biggest issues with Airbnb is getting the occupancy rate for each host or for a market. *Inside Airbnb*, the website I sourced the data from, uses an occupancy model which they call the "San Francisco Model" with the following methodology:

1. A **Review Rate** of 50% is used to convert reviews to estimated bookings. Other administrative authorities are said to use a review rate of 72% (however this may be attributed to an unreliable source: Airbnb's CEO and co-founder Brian Chesky) - or one of 30.5% (based on comparing public data of reviews to the The New York Attorney General’s report on Airbnb released in October 2014.) *Inside Airbnb* chose 50% as it sits almost exactly between 72% and 30.5%. It basically means that only 50% of all visitors write a review. With that said, the number of reviews per month divided by the review rate equals an estimate of actual visitors.
2. An **average length of stay** for each city is usually published by Airbnb. This number multiplied by the estimated bookings for each listing over a period of time gives **the occupancy rate**.
3. Finally, the **income** can be calculated by multiplying the occupancy rate by the price and the time period of interest - here, 12 months:

>**Monthly Occupancy Rate = Average Length of Stay * (No. of reviews per Month / Review Rate)**<br>
*According to the latest Airbnb update, guests who booked stays in Berlin in 2017 via Airbnb spent 4.2 nights here on average.*

>**Yearly Income = Monthly Occupancy Rate * Price * 12 Months**

*Modest Estimate*

With a very modest review rate of 0.5, we are assuming that only every second guest left a review. While I could see many more than just half the visitors actually writing feedback, let's calculate this conservative estimate first:

In [ ]:
avg_length_of_stay_berlin = 4.2
review_rate_modest = 0.5

# calculate the occupancy and round the result
df['modest_occupancy'] = round(avg_length_of_stay_berlin * (df['reviews_per_month']/review_rate_modest), 2)

# occupancy cannot be greater than 100% - are there any mistakes?
len(df[df['modest_occupancy'] > 100])

In [ ]:
# let's drop occupancy rates > 100
df.drop(df[(df['modest_occupancy'] > 100)].index, axis=0, inplace=True)

# check the distribution
df['modest_occupancy'].describe()

*Optimistic Estimate*

Now let's try for a more optimistic estimate of occupancy using a review rate of 0.4, which assumes that only 40% of all guests left a review. This way, the number of reviews points to a higher occupancy than in the modest estimate we had before:

In [ ]:
review_rate_optimistic = 0.4

# calculate the occupancy and round the result
df['optimistic_occupancy'] = round(avg_length_of_stay_berlin * (df['reviews_per_month']/review_rate_optimistic), 2)

# occupancy cannot be greater than 100% - are there any mistakes?
len(df[df['optimistic_occupancy'] > 100])

In [ ]:
# let's drop the occupancy rates > 100
df.drop(df[(df['optimistic_occupancy'] > 100)].index, axis=0, inplace=True)

# check the distribution
df['optimistic_occupancy'].describe()

*Comparing Occupancy Estimates*

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(8,5))

sns.distplot(df['modest_occupancy'], kde=False, bins=20, color='dimgrey')
sns.distplot(df['optimistic_occupancy'], kde=False, bins=20, color='gold')

plt.title('\nAverage Occupancy Rate\n', fontweight='bold')
plt.legend(['Modest Estimate', 'Optimistic Estimate'])
plt.xlabel('\nMonthly Occupancy in %')
plt.ylabel('Proportion of Accommodations\n')

# draw and label a line with median value of modest estimate
plt.axvline(x=14, ymin=0, ymax=1, linewidth=1, linestyle=':', color='black')
plt.text(14.5, 5350, 'Median (Modest Estimate)')

# draw and label a line with median value of optimistic estimate
plt.axvline(x=17.5, ymin=0, ymax=0.9, linewidth=1, linestyle='-', color='gold')
plt.text(18, 4500, 'Median (Optimistic Estimate)', color='goldenrod');

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.5. Income Estimate
<a id='2.5. Income Estimate'></a>

Using our modest and optimistic estimates for the occupancy rate, we'll now do the same for income:

In [ ]:
df['modest_income'] = df['modest_occupancy'] * df['price'] * 12
df.modest_income.describe()

In [ ]:
df['optimistic_income'] = df['optimistic_occupancy'] * df['price'] * 12
df.optimistic_income.describe()

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(8,5))

sns.distplot(df['modest_income'], kde=False, bins=100, color='dimgrey')
sns.distplot(df['optimistic_income'], kde=False, bins=100, color='crimson')

plt.title('\nAverage Yearly Income\n', fontweight='bold')
plt.legend(['Modest Estimate', 'Optimistic Estimate'])
plt.xlabel('\nYearly Income in €')
plt.xlim((0,100000))
plt.ylabel('Proportion of Accommodations\n')

# draw and label a line with median value of modest estimate
plt.axvline(x=8709, ymin=0, ymax=1, linewidth=1, linestyle=':', color='black')
plt.text(9350, 2650, 'Median (Modest Estimate)')

# draw and label a line with median value of optimistic estimate
plt.axvline(x=10886, ymin=0, ymax=0.9, linewidth=1, linestyle='-', color='firebrick')
plt.text(12000, 2350, 'Median (Optimistic Estimate)', color='firebrick');

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 3. Exploratory Data Analysis (EDA)
<a id='3. Exploratory Data Analysis (EDA)'></a>

#### 3.1. Getting Started with a Heat Map
<a id='3.1. Getting Started with a Heat Map'></a>

In [ ]:
sns.set(style="white")
corr = df.corr()

# generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# set up the matplotlib figure
fig, ax = plt.subplots(figsize=(8, 6))

# generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink":.5});

There is reason to believe that 
- a) the number of people that can be `accommodated` as an indicator for **size or capacity** and 
- b) the `latitude` as a proxy for **location** may help explaining the demand. 

(The feature `reviews_per_month` is what we used to estimate the occupancy, so no wonder the correlation is a vivid red.)

I assume there might be also some **seasonality** that strongly influences the demand. Furthermore, I believe that the so-called **super host status** does benefit these hosts, who probably get far more guests than standard hosts. As the heat map only uses numeric columns, it can't show any such relationship — so let's walk through all of these factors to visually investigate their effect on demand:

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 3.2. Seasonal Demand
<a id='3.2. Seasonal Demand'></a>

To examine if there's an underlying seasonality, let's split the dataset we loaded at the very beginning by year. We'll repeat the exact same steps we used earlier to pull out the full calendar year.

In [ ]:
# split the data by date
count_2018 = df_raw[df_raw["date"].isin(pd.date_range('2018-01-01', '2018-10-31'))]
count_2017 = df_raw[df_raw["date"].isin(pd.date_range('2017-01-01', '2017-12-31'))]
count_2016 = df_raw[df_raw["date"].isin(pd.date_range('2016-01-01', '2016-12-31'))]
count_2015 = df_raw[df_raw["date"].isin(pd.date_range('2015-01-01', '2015-12-31'))]

# set the date as index and sort by index
count_2018 = count_2018.set_index('date').sort_index()
count_2017 = count_2017.set_index('date').sort_index()
count_2016 = count_2016.set_index('date').sort_index()
count_2015 = count_2015.set_index('date').sort_index()

# calculate reviews per month                       
count_2018_monthly = count_2018.groupby([pd.TimeGrouper(freq='M'), 'listing_id'])['review_count']\
                                         .agg({'review_count':{'reviews_per_month_18':'count'}})\
                                         .reset_index()
count_2017_monthly = count_2017.groupby([pd.TimeGrouper(freq='M'), 'listing_id'])['review_count']\
                                         .agg({'review_count':{'reviews_per_month_17':'count'}})\
                                         .reset_index()
count_2016_monthly = count_2016.groupby([pd.TimeGrouper(freq='M'), 'listing_id'])['review_count']\
                                         .agg({'review_count':{'reviews_per_month_16':'count'}})\
                                         .reset_index()
count_2015_monthly = count_2015.groupby([pd.TimeGrouper(freq='M'), 'listing_id'])['review_count']\
                                         .agg({'review_count':{'reviews_per_month_15':'count'}})\
                                         .reset_index()

# flatten the multi-index
count_2018_monthly.columns = ['date', 'listing_id', 'reviews_per_month_18']
count_2017_monthly.columns = ['date', 'listing_id', 'reviews_per_month_17']
count_2016_monthly.columns = ['date', 'listing_id', 'reviews_per_month_16']
count_2015_monthly.columns = ['date', 'listing_id', 'reviews_per_month_15']

# set date as index
count_2018_monthly.set_index('date', inplace=True)
count_2017_monthly.set_index('date', inplace=True)
count_2016_monthly.set_index('date', inplace=True)
count_2015_monthly.set_index('date', inplace=True)

In [ ]:
sns.set(style="dark")
fig, axes = plt.subplots(4, 1, figsize=(12,10))

sub_1 = count_2018_monthly.groupby('date')['reviews_per_month_18'].mean()
sub_1.plot(ax=axes[0], color='midnightblue', style=':')
axes[0].set_title('2018', fontweight='bold')
axes[0].set_xlabel('') 

sub_2 = count_2017_monthly.groupby('date')['reviews_per_month_17'].mean()
sub_2.plot(ax=axes[1], color='grey', style=':')
axes[1].set_title('2017', fontweight='bold')
axes[1].set_xlabel('') 

sub_3 = count_2016_monthly.groupby('date')['reviews_per_month_16'].mean()
sub_3.plot(ax=axes[2], color='coral', style=':')
axes[2].set_title('2016', fontweight='bold')
axes[2].set_xlabel('') 

sub_4 = count_2015_monthly.groupby('date')['reviews_per_month_15'].mean()
sub_4.plot(ax=axes[3], color='forestgreen', style=':')
axes[3].set_title('2015', fontweight='bold')
axes[3].set_xlabel('') 

# adjust space between subplots and set a title
plt.subplots_adjust(hspace = 0.6)
plt.suptitle('\nAverage Reviews per Month for Berlin\n', fontweight='bold')

# plot common y-label
fig.text(0.04, 0.5, 'Average Reviews per Month', fontweight='bold', va='center', rotation='vertical');

Be aware that - unlike the other plots - the one for 2018 only ranges from January to October! 

Having said that, we can see the same pattern each and every year: the visitors peak from May to July, and then again in September and October. It drops significantly during August and the winter months. The pattern in August is interesting: tourists seem to avoid city trips in the, liekely preferring to go on beach holidays.

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 3.3. Demand by Status SUPERHOST
<a id='3.3. Demand by Status SUPERHOST'></a>

In [ ]:
# check super host column for NaN's
df.host_is_superhost.isna().sum()

In [ ]:
# replace NaN's with value 'false' for not being a superhost
df.host_is_superhost.fillna(value='f', inplace=True)
df.host_is_superhost.isna().sum()

*What do the differences between super hosts and standard hosts look like?*

In [ ]:
# assign the data
super_host    = df.host_is_superhost.value_counts()['t']
no_super_host = df.host_is_superhost.value_counts()['f']
names = ['Super Hosts','Standard Hosts']
size  = [super_host, no_super_host]

# create a pie chart
plt.pie(size, labels=names, colors=['darkorange','silver'], 
        autopct='%.2f%%', pctdistance=1.28,
        wedgeprops={'linewidth':7, 'edgecolor':'white'})

# create circle for the center of the plot to make the pie look like a donut
my_circle = plt.Circle((0,0), 0.6, color='white')

# plot the donut chart
fig = plt.gcf()
fig.set_size_inches(7,7)
fig.gca().add_artist(my_circle)
plt.show()

In [ ]:
# the overall median differences between superhosts and standard hosts
df.groupby('host_is_superhost')['modest_occupancy'].agg(np.median)

In [ ]:
# group by neighbourhood and by status, then take the median occupancy
df_grouped = pd.DataFrame(df.groupby(['neighbourhood', 'host_is_superhost'])['modest_occupancy'].agg(np.median))
df_grouped.sort_values(by='modest_occupancy', ascending=False, inplace=True)
df_grouped.reset_index(inplace=True)

# plot grouped dataframe with seaborn
sns.set_style('darkgrid')
fig, ax = plt.subplots(figsize=(10, 9))
sns.barplot(x='modest_occupancy', y='neighbourhood', hue='host_is_superhost', hue_order=['t', 'f'], 
            data=df_grouped, palette="Blues_r")

# add title and axis label
ax.set_title('\nOccupancy by Area and Host Status\n', fontsize=14, fontweight='bold')
ax.set(xlabel='Average Occupancy in % (Modest Estimate)')

# rename legend labels & put legend outside plot
handles, labels = ax.get_legend_handles_labels()
ax.legend((handles), ('Super Host', 'Normal Host'), bbox_to_anchor=(1.05, 1), borderaxespad=0.);

In [ ]:
# the overall median differences between superhosts and standard hosts
df.groupby('host_is_superhost')['modest_income'].agg(np.median)

In [ ]:
# group by neighbourhood and by status, then take the median income
df_grouped = pd.DataFrame(df.groupby(['neighbourhood', 'host_is_superhost'])['modest_income'].agg(np.median))
df_grouped.sort_values(by='modest_income', ascending=False, inplace=True)
df_grouped.reset_index(inplace=True)

# plot grouped dataframe with seaborn
sns.set_style('dark')
fig, ax = plt.subplots(figsize=(10, 9))
sns.barplot(x='modest_income', y='neighbourhood', hue='host_is_superhost', hue_order=['t', 'f'], 
            data=df_grouped, palette="Purples_r")

# add title and axis label
ax.set_title('\nIncome by Area and Host Status\n', fontsize=14, fontweight='bold')
ax.set(xlabel='\nAverage Income in € (Modest Estimate)')

# rename legend labels & put legend outside plot
handles, labels = ax.get_legend_handles_labels()
ax.legend((handles), ('Super Host', 'Normal Host'), bbox_to_anchor=(1.25, 1), borderaxespad=0.);

It's definitely worth aspring to become a super host! The differences in occupancy and income are striking!

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 3.4. Demand by Neighbourhood
<a id='3.4. Demand by Neighbourhood'></a>

*Lollipop Charts with two observations*

In [ ]:
def lollipop_with_2(dataframe, about, y_col, x1_col, x2_col):
    ''' Function that creates a lollipop chart with 
    two observations (x-values) per group (y-value). '''
       
    # set style
    sns.set_style("dark")
    
    # prepare data
    df_plot = dataframe[[y_col, x1_col, x2_col]].groupby(y_col).apply(lambda x: x.mean())
    df_plot.sort_values(x1_col, inplace=True)
    df_plot.reset_index(inplace=True)
        
    # draw plot
    fig, ax = plt.subplots(figsize=(10,8))
    ax.hlines (y=df_plot.index, xmin=df_plot[x1_col], xmax=df_plot[x2_col], color='grey', alpha=0.7, linewidth=2)
    ax.scatter(y=df_plot.index, x=df_plot[x1_col], s=75, color='skyblue', alpha=0.7, label=x1_col)
    ax.scatter(y=df_plot.index, x=df_plot[x2_col], s=75, color='darkblue', alpha=0.7, label=x2_col)
    ax.legend()
    
    # title, label, ticks and xlim
    ax.set_title('\nAverage ' + about + ' by District\n', fontdict={'size':12, 'weight':'bold'})
    ax.set_xlabel('\n' + about)
    ax.set_yticks(df_plot.index)
    ax.set_yticklabels(df_plot[y_col], rotation=0, 
                       fontdict={'horizontalalignment': 'right', 'size':12})
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [ ]:
lollipop_with_2(df, 'Occupancy Estimate in %', 'neighbourhood', 'modest_occupancy', 'optimistic_occupancy')

Unsurprisingly, *Mitte* is the most in-demand area; it's a hotspot, right in the center, and close to most places of interest. But why the heck is *Reinickendorf* at the top?

In [ ]:
lollipop_with_2(df, 'Income Estimate in €', 'neighbourhood', 'modest_income', 'optimistic_income')

It appears to be that *Reinickendorf* benefits from rather low room rates, which lead to a high occupancy - as seen above. When it comes to earnings, Mitte is in the lead.

In [ ]:
lollipop_with_2(df, 'Fees in €', 'neighbourhood', 'cleaning_fee', 'price')

*Lollipop Charts with one observation and one group highlighted*

In [ ]:
def lollipop_with_highlight(dataframe, about, y_col, y_highlight, x_col):
    ''' Function that creates a lollipop chart with only one observation (x-value) 
    per group (y-value) and the option to highlight one of the grouped values. '''
    
    # set style
    sns.set_style("dark")
    
    # prepare data
    df_plot = dataframe[[y_col, x_col]].groupby(y_col).apply(lambda x: x.mean())
    df_plot.sort_values(x_col, inplace=True)
    df_plot.reset_index(inplace=True)

    # highlight district specified as y_highlight
    my_color = np.where(df_plot[y_col] == y_highlight, 'crimson', 'skyblue')
    my_size  = np.where(df_plot[y_col] == y_highlight, 70, 30)

    # draw plot
    fig, ax = plt.subplots(figsize=(8,8))
    ax.hlines (y=df_plot.index, xmin=0, xmax=df_plot[x_col], color=my_color, alpha=0.5, linewidth=2)
    ax.scatter(y=df_plot.index, x=df_plot[x_col], s=my_size, color=my_color, alpha=1, label=x_col)
    ax.legend()

    # title, label, ticks and xlim
    ax.set_title('\nAverage ' + about + ' by District\n', fontdict={'size':12, 'weight':'bold'})
    ax.set_xlabel('\n' + about)
    ax.set_yticks(df_plot.index)
    ax.set_yticklabels(df_plot[y_col], rotation=0, 
                       fontdict={'horizontalalignment': 'right', 'size':12})
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [ ]:
lollipop_with_highlight(df, 'Occupancy Estimate', 'neighbourhood', 'Mitte', 'optimistic_occupancy')

In [ ]:
lollipop_with_highlight(df, 'Income Estimate', 'neighbourhood', 'Mitte', 'optimistic_income')

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 3.5. Demand by Capacity
<a id='3.5. Demand by Capacity'></a>

In [ ]:
sns.set_style("darkgrid")
fig, ax = plt.subplots(figsize=(9,6))

sns.scatterplot(x="accommodates", y="modest_occupancy", color='rosybrown',  data=df)

plt.title('\nAverage Occupancy by Capacity\n', fontsize=12, fontweight='bold')
plt.xlabel('\nNumber of Max. People per Accommodation')
plt.ylabel('Occupancy (Modest Estimate)\n');

In [ ]:
# group by accomodates and plot median occupancy
df.groupby('accommodates')['modest_occupancy']\
                          .median()\
                          .plot(kind='bar', figsize=(10,6), color='rosybrown')

# beautify the plot
plt.xlabel('\nNumber of Max. People per Accommodation', fontsize=12)
plt.xticks(rotation='horizontal')
plt.ylabel('Average Occupancy in %\n', fontsize=12)
plt.title('\nAverage Occupancy by Capacity\n', fontsize=12, fontweight='bold')

# draw and label the median value
plt.axhline(y=14.7, xmin=0, xmax=1, linewidth=1, linestyle=':', color='black')
plt.text(x=15.6, y=14.2, s='Overall Median (Modest Estimate)');

Generally, bigger homes seem to be sold more often than smaller ones. Perhaps this is due to the fact that a group might be able to save more money than 1-2 persons would by using Airbnb. It follows that accommodations with a bigger capacity enjoy greater popularity.

In [ ]:
# prepare plot
sns.set_style("white")
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)
fig, ax = plt.subplots(figsize=(11,7))

# draw scatter plot
ax = sns.scatterplot(x="longitude", y="latitude", size='modest_occupancy', sizes=(5, 200),
                     hue='accommodates', palette=cmap,  data=df)
ax.legend(bbox_to_anchor=(1.3, 1), borderaxespad=0.)
plt.title('\nAccommodations in Berlin by Capacity & Occupancy\n', fontsize=12, fontweight='bold')

# remove spines
sns.despine(ax=ax, top=True, right=True, left=True, bottom=True);

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 4. Interpreting the Data
<a id='4. Interpreting the Data'></a>

**> Seasonality**

The high season for lodgings in Berlin is during late spring and early summer (specifically from May to July) and during the autumn months September and October. You could use your apartment yourself during the rest of year and offer it in these periods to get the most out of it.

**> Super Host**

To be a superhost is to be a cash machine. The occupancy rate in superhost lodgings is almost twice as high as in standard host lodgings, and the income is 60% higher.

**> Location, Location, Location**

If you don't want to live in the vibrant, loud center of Berlin yourselves - well, that's bad luck for you! That's precisely where tourists are looking to to rent Airbnb accommodations, particularly in *Mitte* or *Charlottenburg*, 
and are willing to pay more for them than for lodgings in outlying districts.

**> Capacity**

Travelers in (bigger) groups benefit much more from how much they save by using Airbnb than couples or small groups do. That implies that lodgings accommodating 6+ people tend to be more in-demand than smaller ones.

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 5. Appendix 
<a id='5. Appendix'></a>

All resources used in this notebook are listed below.

Data
- Inside Airbnb: http://insideairbnb.com/get-the-data.html

Occupancy Model
- http://insideairbnb.com/about.html
- https://www.airbnbcitizen.com/hosts-on-airbnb-have-welcomed-700000-guests-to-berlin-in-2017/
- https://www.airbnbcitizen.com/wp-content/uploads/2016/04/airbnb-community-berlin-en.pdf

Lollipop Plots
- https://python-graph-gallery.com/184-lollipop-plot-with-2-groups/
- https://python-graph-gallery.com/183-highlight-a-group-in-lollipop/
- https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python/#16.-Lollipop-Chart

Bubble Plots
- https://glowingpython.blogspot.com/2011/11/how-to-make-bubble-charts-with.html
- https://seaborn.pydata.org/examples/scatterplot_sizes.html

Inspiration
- https://towardsdatascience.com/airbnb-rental-listings-dataset-mining-f972ed08ddec
- https://towardsdatascience.com/improving-airbnb-yield-prediction-with-text-mining-9472c0181731
- https://www.kaggle.com/yogi045/how-to-become-top-earner-in-airbnb?utm_medium=email&utm_source=intercom&utm_campaign=datanotes-2019